# Árboles de decisión y Random Forests

## 1. Introducción teórica a los árboles simples

Los métodos de machine learning basados en árboles comprenden un conjunto de técnicas de aprendizaje supervisado, que permiten segmentar el espacio de los predictores en regiones simples, dentro de las cuales es más sencillo manejar las interacciones y predecir la variable respuesta. Dado que el conjunto de reglas de división utilizadas para segmentar el espacio predictor se puede resumir en un árbol, este tipo de efoques se conocen como métodos de árboles de decisión.

Aunque los métodos basados en árboles de decisión simples tienen ventajas como la simplicidad y la interpretabilidad, no son competitivos en cuanto a precisión si los comparamos con otros enfoques. Es por esto que en este notebook también veremos otros algoritmos basados en árboles como `bagging`, `random forests` o `boosting`. Estos métodos mejoran el desempeño de los árboles simples combinando (*ensemble methods*) varios de ellos, a costa de perder interpretabilidad.

Aunque el porcentaje mayoritario de aplicación es el campo de la clasificación, también se pueden emplear en regresiones de variables continuas.

**Ventajas:**

- Facilidad de interpretación, no es necesario tener conocimientos estadísticos.

- Se pueden graficar e interpretar de forma sencilla mientras que el modelo no tenga más de un árbol.

- Se pueden introducir variables numéricas y categóricas, sin necesidad de dummificar éstas últimas. No obstante, a veces es recomdable categorizar las variables numéricas. 

- Al tratarse de métodos no paramétricos, no es necesario que se cumpla ningún tipo de distribución específica en los datos.

- Requieren menos limpieza y trabajo de preprocesado que otros métodos.

- Son robustos frente a la presencia de outliers.

- Son muy útiles en la exploración de datos, permiten identificar de forma rápida y eficiente las variables más importantes.

- Son capaces de seleccionar predictores de forma automática.

**Desventajas:**

- El árbol de decisión tiene tendencia al overfitting. Este inconveniente se puede mitigar combinando múltiples árboles y obteniendo valores promedio (bagging, random forests, boosting).

- Debido a la categorización de las variables continuas, suelen obtener mejores resultados en ejercicios de clasificación que en los de regresión.

- Tal y como se describe más adelante, la creación de las ramificaciones de los árboles se consigue mediante el algoritmo de recursive binary splitting. Este algoritmo identifica y evalúa las posibles divisiones de cada predictor acorde a una determinada medida (RSS, Gini, entropía…). Los predictores continuos o predictores cualitativos con muchos niveles tienen mayor probabilidad de contener, solo por azar, algún punto de corte óptimo, por lo que suelen verse favorecidos en la creación de los árboles.

### 1.1. Árboles de regresión

#### 1.1.1. Idea intuitiva    

Los árboles de regresión son un subtipo de árboles de predicción que trabaja con variables numéricas continuas, y que tratan de predecir por tanto un valor númerico de salida, en base a las diferentes variables dependientes del modelo.

#### 1.1.2. Construcción 

El proceso de construcción de un árbol de regresión se puede dividir en dos etapas:

1. División sucesiva del espacio de los predictores $X_1$, $X_2$, ..., $X_p$ en $J$ regiones no solapantes (nodos terminales) $R_1$, $R_2$, ..., $R_j$. Aunque, desde el punto de vista teórico las regiones podrían tener cualquier forma, si se limitan a regiones rectangulares (de múltiples dimensiones), se simplifica en gran medida el proceso de construcción y se facilita la interpretación.

2. Predicción de la variable respuesta en cada región. Para cada observación que caiga dentro de una región $R_j$, se le asigna siempre el mismo valor, el cual es simplemente la media de los valores de respuesta contenidos en la región $R_j$.

A pesar de la sencillez con la que se puede resumir el proceso de construcción de un árbol, es necesario establecer una metodología que permita crear las regiones $R_1$, $R_2$, ..., $R_j$, o lo que es equivalente, decidir donde se introducen las divisiones: en que predictores y en que valores de los mismos. Es en este punto donde se diferencian los algoritmos de árboles de regresión frente a los de clasificación.

En el caso de los árboles de regresión, el criterio más frecuentemente empleado para identificar las divisiones es el Residual Sum of Squares (RSS). El objetivo es encontrar las $J$ regiones que minimizan el Residual Sum of Squares (RSS) total:

\begin{align}
RSS =\sum_{j=1}^J \sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2
\end{align}

donde $\hat{y}_{R_j}$ es la media de la variable respuesta en la región $R_j$. Una descripción menos técnica equivale a decir que se busca una distribución de regiones tal que, el sumatorio de las desviaciones al cuadrado entre las observaciones y la media de la región a la que pertenecen sea lo menor posible.

Desafortunadamente, no es posible considerar todas las posibles particiones del espacio de los predictores. Por esta razón, se recurre a lo que se conoce como *recursive binary splitting* (división binaria recursiva). Este enfoque comienza en la parte superior del árbol (en cuyo punto todas las observaciones pertenecen a una sola región) y luego divide sucesivamente el espacio predictor; cada división se indica a través de dos nuevas ramas en el árbol. Como la división se realiza en ese paso en particular, en lugar de mirar hacia adelante y elegir una división que conducirá a un mejor árbol en algún paso futuro, este tipo de enfoques se denominan *greedy* (codiciosos). Este método no evalúa todas las posibles regiones pero, alcanza un buen balance computación-resultado.

**Recursive Binary Splitting:**

El objetivo del método recursive binary splitting es encontrar en cada iteración, el predictor $X_j$ y el punto de corte (umbral) $s$, tal que si se distribuyen las observaciones en las regiones $\lbrace X | X_j < s \rbrace$ y $\lbrace X | X_j ≥ s \rbrace$, se consigue la mayor reducción posible del RSS. Los pasos del algoritmo son los siguientes:

1. El proceso se inicia en lo más alto del árbol, donde todas las observaciones pertenecen a la misma región.

2. Se identifican todos los posibles puntos de corte (umbrales) $s$ para cada uno de los predictores ($X_1$, $X_2$, …, $X_p$). En el caso de predictores cualitativos, los posibles puntos de corte son cada uno de sus niveles. Para predictores continuos, se ordenan de menor a mayor sus valores, el punto intermedio entre cada par de valores se emplea como punto de corte.

3. Se calcula el RSS total que se consigue con cada posible división identificada en el paso 2, donde el primer término es el RSS de la región 1 y el segundo término es el RSS de la región 2, siendo cada una de las regiones el resultado de separar las observaciones acorde al predictor $j$ y valor $s$.

\begin{align}
RSS = RSS_1 + RSS_2 = \sum_{i:x_i \in R_1(j, s)} (y_i - \hat{y}_{R_1})^2 \ + \sum_{i:x_i \in R_2(j, s)} (y_i - \hat{y}_{R_2})^2
\end{align}

4. Se selecciona el predictor $X_j$ y el punto de corte $s$ que resulta en el menor RSS total, es decir, que da lugar a las divisiones más homogéneas posibles. Si existen dos o más divisiones que consiguen la misma mejora, la elección entre ellas es aleatoria.

5. Se repiten de forma iterativa los pasos 1 a 4 para cada una de las regiones que se han creado en la iteración anterior hasta que se alcanza alguna norma de stop. Algunas de las más empleadas son: que ninguna región contenga un mínimo de $n$ observaciones, que el árbol tenga un máximo de nodos terminales o que la incorporación del nodo reduzca el error en al menos un % mínimo.

Está metodología conlleva dos hechos:

- Que cada división óptima se identifica acorde al impacto que tiene en ese momento. No se tiene en cuenta si es la división que dará lugar a mejores árboles en futuras divisiones (método *greedy*).

- En cada división se evalúa un único predictor haciendo preguntas binarias (¿sí?, ¿no?), lo que genera dos nuevas ramas del árbol por división. A pesar de que es posible evaluar divisiones más complejas, hacer una pregunta sobre múltiples variables a la vez es equivalente a hacer múltiples preguntas sobre variables individuales.

#### 1.1.3. Predicción

Tras la creación de un árbol, las observaciones de entrenamiento quedan agrupadas en los nodos terminales. Para predecir una nueva observación, se recorre el árbol en función de los valores que tienen sus predictores hasta llegar a uno de los nodos terminales. En el caso de árboles de regresión, el valor predicho suele ser la media de la variable respuesta de las observaciones de entrenamiento que están en ese mismo nodo. Si bien la media es valor más empleado, también se podría utilizar cualquier otro (mediana, cuantil, …etc).

### 1.2. Árboles de clasificación

#### 1.2.1. Idea intuitiva    

Los árboles de clasificación se asemejan mucho a los árboles de regresión, con la diferencia de que la variable salida del modelo es cualitativa, en vez de continua.

#### 1.2.2. Construcción 

Para construir un árbol de clasificación se emplea el mismo método de recursive binary splitting descrito en los árboles de regresión. Sin embargo, como la variable respuesta es cualitativa, no es posible emplear el RSS como criterio de selección de las divisiones óptimas. Existen varias alternativas, todas ellas con el objetivo de encontrar nodos lo más puros u homogéneos posible. Las más empleadas son:

**Classification Error Rate:**

Se define como la proporción de observaciones que no pertenecen a la clase más común dentro del nodo.

\begin{align}
E_m = 1 -  max_k(\hat{p}_{mk})
\end{align}

donde $\hat{p}_{mk}$ representa la proporción de observaciones del nodo $m$ que pertenecen a la clase $k$. A pesar de la sencillez de esta medida, no es suficientemente sensible para crear buenos árboles, por lo que, en la práctica, suelen emplearse otras medidas.

**Gini Index:**

Es una medida de la varianza total en el conjunto de las $K$ clases del nodo $m$. Se considera una medida de pureza del nodo.

\begin{align}
G_m = \sum_{k = 1}^K \hat{p}_{mk}(1 - \hat{p}_{mk})
\end{align}

Cuando $\hat{p}_{mk}$ es cercano a 0 o a 1 (el nodo contiene mayoritariamente observaciones de una clase), el término $\hat{p}_{mk}(1 - \hat{p}_{mk})$ es muy pequeño. Como consecuencia, cuanto mayor sea la pureza del nodo, menor será el valor del índice Gini G.

El algoritmo CART (Classification and regression trees) emplea el Gini Index como criterio de división.

**Information Gain - Cross Entropy**:

La entropía es otra forma de cuantificar el desorden de un sistema. En el caso de los nodos, el desorden se corresponde con la impureza.

Si un nodo es puro, contiene únicamente observaciones de una clase, entonces su entropía es cero. Por el contrario, si la frecuencia de cada clase es la misma, el valor de la entropía alcanza el valor máximo de 1.

\begin{align}
D = - \sum_{k = 1}^K \hat{p}_{mk}log(\hat{p}_{mk})
\end{align}

Los algoritmos C4.5 y C5.0 emplean information gain como criterio de división.

**Chi - Square ($\chi^2$)**:

Esta aproximación consiste en identificar si existe una diferencia significativa entre los nodos hijos y el nodo padre, es decir, si hay evidencias de que la división consigue una mejora efectiva. Para ello, se aplica un test estadístico *chi-square goodness of fit* empleando como distribución esperada $H_0$ la frecuencia de cada clase en el nodo parental. Cuanto mayor sea el estadístico $\chi^2$, mayor es la evidencia estadística de que existe una diferencia.

\begin{align}
\chi^2 = - \sum_{k} \frac{(observado_k - esperado_k)^2}{esperado_k}
\end{align}

Los árboles generados con este criterio de división reciben el nombre de CHAID (Chi-square Automatic Interaction Detector).

Independientemente de la medida empleada como criterio de selección de las divisiones, el proceso siempre es el mismo:

1. Para cada posible división se calcula el valor de la medida en cada uno de los dos nodos resultantes.

2. Se suman los dos valores ponderando cada uno por la fracción de observaciones que contiene cada nodo. Este paso es muy importante, ya que no es lo mismo dos nodos puros con 2 observaciones, que dos nodos puros con 100 observaciones.

\begin{align}
\frac{n\ observaciones\ nodo\ A}{n\ observaciones\ totales} · pureza\ A + \frac{n\ observaciones\ nodo\ B}{n\ observaciones\ totales} · pureza\ B 
\end{align}

3. La división con menor o mayor valor (dependiendo de la medida empleada) se selecciona como división óptima.

Para el proceso de construcción del árbol, acorde al libro *Introduction to Statistical Learning*, Gini Index y cross-entropy son más adecuados que el classification error rate debido a su mayor sensibilidad a la homogeneidad de los nodos. 

Para el proceso de pruning (descrito en la siguiente sección) los tres son adecuados, aunque, si el objetivo es conseguir la máxima precisión en las predicciones, mejor emplear el classification error rate.

#### 1.2.3. Predicción

Tras la creación de un árbol, las observaciones de entrenamiento quedan agrupadas en los nodos terminales. Para predecir una nueva observación, se recorre el árbol en función del valor de sus predictores hasta llegar a uno de los nodos terminales. En el caso de clasificación, suele emplearse la moda de la variable respuesta como valor de predicción, es decir, la clase más frecuente del nodo. Además, puede acompañarse con el porcentaje de cada clase en el nodo terminal, lo que aporta información sobre la confianza de la predicción.

### 1.3. Overfitting en los árboles simples

El proceso de construcción de árboles descrito en las secciones anteriores tiende a reducir rápidamente el error de entrenamiento, es decir, el modelo se ajusta muy bien a las observaciones empleadas en el conjunto de entrenamiento. Como consecuencia, se genera un overfitting que reduce su capacidad predictiva al aplicarlo a nuevos datos.

La razón de este comportamiento radica en la facilidad con la que los árboles se ramifican adquiriendo estructuras complejas. De hecho, si no se limitan las divisiones, todo árbol termina ajustándose perfectamente a las observaciones de entrenamiento creando un nodo terminal por observación.

Existen dos estrategias para prevenir el problema de overfitting de los árboles:

- Limitación del tamaño del árbol
- Podado del árbol (pruning)

#### 1.3.1. Controlar el tamaño del árbol

El tamaño final que adquiere un árbol puede controlarse mediante reglas de parada que detengan la división de los nodos dependiendo de si se cumplen o no determinadas condiciones. El nombre de estas condiciones puede variar dependiendo del software o librería empleada, pero suelen estar presentes en todos ellos.

- Observaciones mínimas para división: define el número mínimo de observaciones que debe tener un nodo para poder ser dividido. Cuanto mayor el valor, menos flexible es el modelo.

- Observaciones mínimas de nodo terminal: define el número mínimo de observaciones que deben tener los nodos terminales. Su efecto es muy similar al de observaciones mínimas para división.

- Profundidad máxima del árbol: define la profundidad máxima del árbol, entendiendo por profundidad máxima el número de divisiones de la rama más larga (en sentido descendente) del árbol.

- Número máximo de nodos terminales: define el número máximo de nodos terminales que puede tener el árbol. Una vez alcanzado el límite, se detienen las divisiones. Su efecto es similar al de controlar la profundidad máxima del árbol.

- Reducción mínima de error: define la reducción mínima de error que tiene que conseguir una división para que se lleve a cabo.

Todos estos parámetros es lo que se denomina como hiperparámetros, son variables que no se aprenden durante el entrenamiento del modelo, y tienen que ser definidas por el usuario en base a su conocimiento del problema y mediante el uso de validación cruzada.

#### 1.3.2. Tree Pruning - Podado

La estrategia de controlar el tamaño del árbol mediante reglas de parada tiene un inconveniente, el árbol crece seleccionando la mejor división en cada momento hasta alcanzar una condición de parada. Al evaluar las divisiones sin tener en cuenta las que vendrán después, nunca se elige la opción que resulta en el mejor árbol final, a no ser que también sea la que genera en ese momento la mejor división. A este tipo de estrategias se les conoce como greedy.

Un ejemplo que ilustra el problema de este tipo de enfoques es el siguiente: supóngase que un coche circula por el carril izquierdo de una carretera que tiene dos carriles por sentido. En el carril que se encuentra hay muchos coches circulando a 100 km/h, mientras que el otro carril se encuentra vacío. A cierta distancia se observa que hay un vehículo circulando por el carril derecho a 20 km/h. Si el objetivo del conductor es llegar a su destino lo antes posible tiene dos opciones: cambiarse de carril o mantenerse en el que está. Una aproximación de tipo greedy evaluaría la situación en ese instante y determinaría que la mejor opción es cambiarse de carril y acelerar a más de 100 km/h, sin embargo, a largo plazo, esta no es la mejor solución, ya que una vez alcance al vehículo lento, tendrá que reducir mucho su velocidad.

Una alternativa no greedy que consigue evitar el overfitting consiste en generar árboles grandes, sin condiciones de parada más allá de las necesarias por las limitaciones computacionales, para después podarlos (pruning), manteniendo únicamente la estructura robusta que consigue un error bajo cuando se utiliza el conjunto de datos de test. La selección del sub-árbol óptimo puede hacerse mediante cross-validation, sin embargo, dado que los árboles crecen lo máximo posible (tienen muchos nodos terminales) no suele ser viable estimar el test error de todas las posibles sub-estructuras que se pueden generar. En su lugar, se recurre al cost complexity pruning o weakest link pruning.

**Cost Complexity Pruning:**

Cost complexity pruning es un método de penalización de tipo *Loss + Penalty*, similar al empleado en *Ridge Regression* o *Lasso*. En este caso, se busca el sub-árbol T que minimize la ecuación:

\begin{align}
\sum_{j=1}^{|T|} \sum_{i \in R_j} (y_i - \hat{y}_{R_j})^2 + \alpha |T|
\end{align}

donde $|T|$ es el número de nodos terminales del árbol.

El primer término de la ecuación se corresponde con el sumatorio total de los residuos cuadrados RSS. Por definición, cuantos más nodos terminales tenga el modelo menor será esta parte de la ecuación.

El segundo término es la restricción, que penaliza al modelo en función del número de nodos terminales (a mayor número, mayor penalización). El grado de penalización se determina mediante el tunning parameter $\alpha$. Cuando $\alpha = 0$, la penalización es nula y el árbol resultante es equivalente al árbol original. A medida que se incrementa $\alpha$ la penalización es mayor, y como consecuencia, los árboles resultantes son de menor tamaño. El valor óptimo de $\alpha$ puede identificarse mediante cross-validation.

El algoritmo para crear un árbolde regresión con *pruning* sería el siguiente:

1. Se emplea *recursive binary splitting* para crear un árbol grande y complejo ($T_0$) empleando los datos de entrenamiento y reduciendo al máximo posible las condiciones de parada. Normalmente se emplea como única condición de parada el número mínimo de observaciones por nodo terminal.

2. Se aplica el *cost complexity pruning* al árbol $T_0$ para obtener el mejor sub-árbol en función de $\alpha$. Es decir, se obtiene el mejor sub-árbol para un rango de valores de $\alpha$.

3. Identificación del valor óptimo de $\alpha$ mediante k-cross-validation. Se divide el training data set en $K$ grupos. Para cada valor de $K$ ($k=1, 2, ..., K$):

    - Repetir los pasos 1 y 2 empleando todas las observaciones excepto las del grupo $k_i$.

    - Evaluar el mean squared error (MSE) para el rango de valores de $\alpha$ empleando el grupo $k_i$.

    - Obtener el promedio de los K-mean-squared-error calculados para cada valor $\alpha$.

4. Seleccionar el sub-árbol del paso 2 que se corresponde con el valor $\alpha$ que ha conseguido el menor cross-validation mean squared error en el paso 3.

En el caso de los árboles de clasificación, en lugar de emplear la suma de residuos cuadrados como criterio de selección, se emplearía alguna de las medidas de homogeneidad vistas en apartados anteriores.

## 2. Introducción teórica a los árboles compuestos - *ensemble methods*

Al igual que todo modelo estadístico, los árboles de predicción sufren el problema del equilibrio sesgo-varianza (*bias-variance tradeoff*). El término sesgo hace referencia a cuánto se alejan en promedio las predicciones de un modelo respecto a los valores reales, es decir, cómo de bien se aproxima el modelo a la relación real entre las variables. La varianza hace referencia a cuánto varía el modelo dependiendo de la muestra empleada en el entrenamiento. A medida que se aumenta la complejidad de un modelo, se dispone de mayor flexibilidad para adaptarlo a las observaciones, reduciendo así el sesgo y mejorando su capacidad predictiva. Sin embargo, alcanzado un determinado grado de flexibilidad, aparece el problema de overfitting, el modelo se ajusta tanto a los datos de entrenamiento que es incapaz de predecir correctamente nuevas observaciones. El mejor modelo es aquel que consigue un equilibro óptimo entre sesgo y varianza.

Por lo general, los árboles pequeños (pocas ramificaciones) no representarán bien la relación entre las variables, por lo que tienen alto bias, pero poca varianza. Los árboles grandes se ajustan mucho a los datos de entrenamiento, por lo que tienen muy poco bias pero mucha varianza.

Los métodos de *ensemble* abarcan un conjunto de técnicas que combinan múltiples modelos predictivos para lograr un equilibro entre sesgo y varianza. Aunque pueden emplearse con multitud de métodos de aprendizaje estadístico (KNN, redes neuronales...etc.), con los árboles de predicción, dan muy buenos resultados. A continuación se describen dos de los más utilizados:

- Bagging: En lugar de ajustar un único árbol, se ajustan muchos de ellos en paralelo formando un “bosque”. En cada nueva predicción, todos los árboles que forman el “bosque” participan aportando su predicción. Como valor final, se toma la media de todas las predicciones (variables continuas) o la clase más frecuente (variables cualitativas). Uno de los métodos de bagging más conocidos es Random Forest.

- Boosting: Consiste en ajustar secuencialmente múltiples modelos sencillos, llamados *weak learners*, de forma que cada modelo aprende de los errores del anterior. Como valor final, al igual que con el algoritmo bagging, se toma la media de todas las predicciones (variables continuas) o la clase más frecuente (variables cualitativas). Tres de los métodos de *Boosting* más empleados son *AdaBoost*, *Gradient Boosting* y *Stochastic Gradient Boosting*.

Aunque el objetivo final es el mismo, lograr un balance óptimo entre sesgo y varianza, existen diferencias importantes entre *Bagging* y *Boosting*:

- La estrategia seguida para reducir el error total: El error total de un modelo puede descomponerse como sesgo + varianza + $\epsilon$. En *Bagging*, se emplean modelos con muy poco sesgo pero mucha varianza, agregando muchos de estos modelos se consigue reducir la varianza sin apenas inflar el sesgo. En *Boosting*, se emplean modelos con muy poca varianza pero mucho sesgo, ajustando secuencialmente muchos modelos se reduce el sesgo.

- Forma en que se crean los distintos modelos que forman el ensemble final. En el caso de *Bagging*, cada modelo es distinto de los otros porque se entrenan con diferentes muestras obtenidas por *bootstrapping* a partir de la muestra original. También se le conoce como *parallel ensemble* porque cada modelo se ajusta independientemente de los otros. En *Boosting*, los modelos se ajustan secuencialmente y la importancia (peso) de las observaciones varía en cada iteración, dando lugar a diferentes ajustes. También se les conoce como sequential ensemble.

La clave para que los métodos de *ensemble* consigan mejores resultados que cualquiera de sus modelos individuales es que, los modelos que los forman, sean lo más diversos posibles. Una analogía que refleja este concepto es la siguiente: supóngase un juego como el trivial en el que los equipos tienen que acertar preguntas sobre temáticas diversas. Un equipo formado por muchos jugadores, cada uno experto en un tema distinto, tendrá más posibilidades de ganar que un equipo formado por jugadores expertos en un único tema o por un único jugador que sabe un poco de todos los temas. A continuación, se explican con detalle cada una de las estrategias.